In [1]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import mrtrix3, utility as niu

In [8]:
def init_mrtrix_wf():
    wf = pe.Workflow(name="mrtrix_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    mrconvert = pe.Node(mrtrix3.MRConvert(), name="mrconvert")
    
    denoise = pe.Node(mrtrix3.DWIDenoise(), name="denoise")
    
    unring = pe.Node(mrtrix3.MRDeGibbs(), name="unring")
    
    resize = pe.Node(mrtrix3.MRResize(voxel_size=(1,1,1)), name="resize")
    
    biascorr = pe.Node(mrtrix3.DWIBiasCorrect(use_ants=True), name="biascorr")
    
    wf.connect([
        (inputnode, mrconvert, [("dwi_file", "in_file"),
                                ("bvec_file", "in_bvec"),
                                ("bval_file", "in_bval")]),
        (mrconvert, denoise, [("out_file", "in_file")]),
        (denoise, unring, [("out_file", "in_file")]),
        (unring, resize, [("out_file", "in_file")]),
        (resize, biascorr, [("out_file", "in_file")]),
        (inputnode, biascorr, [("bvec_file", "in_bvec"),
                               ("bval_file", "in_bval")]),
        (biascorr, outputnode, [("out_file", "out_file")])
        ])
    
    return wf

In [9]:
data_dir = os.path.abspath("../data")
layout = BIDSLayout(data_dir)

In [10]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [11]:
dwi_file = dwi_files[0]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)

In [12]:
test_wf = init_mrtrix_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True

test_wf.run()

190809-13:04:58,895 nipype.workflow INFO:
	 Generated workflow graph: /mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/mrtrix_wf/graph.png (graph2use=colored, simple_form=True).
190809-13:04:59,178 nipype.workflow INFO:
	 Workflow mrtrix_wf settings: ['check', 'execution', 'logging', 'monitoring']
190809-13:04:59,237 nipype.workflow INFO:
	 Running serially.
190809-13:04:59,240 nipype.workflow INFO:
	 [Node] Setting-up "mrtrix_wf.mrconvert" in "/mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/mrtrix_wf/mrconvert".
190809-13:04:59,309 nipype.workflow INFO:
	 [Node] Running "mrconvert" ("nipype.interfaces.mrtrix3.utils.MRConvert"), a CommandLine Interface with command:
mrconvert -fslgrad /mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.bvec /mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dw